# TP Docker - M2DSIA

## I. Créer, lancer et supprimer une image Ubuntu : `daniel-ubuntu`

### 🔹 Étape 1 : Créer le dossier du projet

```bash
mkdir tp-m2dsia-ubuntu
cd tp-m2dsia-ubuntu
```

### 🔹 Étape 2 : Créer le fichier Dockerfile

Créer un fichier `Dockerfile` avec ce contenu :

```Dockerfile
FROM ubuntu:22.04

RUN echo "Je suis à l’étape ….."

ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get update && \
    apt-get install -y curl && \
    apt-get clean

CMD ["bash"]
```

### 🔹 Étape 3 : Construire l’image

```bash
docker build -t daniel-ubuntu .
```

### 🔹 Étape 4 : Lancer un conteneur interactif

```bash
docker run -it --name daniel-container daniel-ubuntu
```

À l’intérieur du conteneur, exécute :

```bash
curl --version
touch test.yaml
ls
```

Puis quitte :

```bash
exit
```

### 🔹 Étape 5 : Supprimer le conteneur et l’image

```bash
docker rm daniel-container
docker rmi daniel-ubuntu
```

---

## II. Comparaison Docker MySQL avec et sans volume

### 🔹 Étape 1 : Créer le dossier du projet

```bash
mkdir tp-mysql-volume
cd tp-mysql-volume
```

### 🔹 Étape 2 : Créer le fichier `init.sql`

Créer un fichier `init.sql` avec le contenu suivant :

```sql
USE testdb;

CREATE TABLE IF NOT EXISTS users_testing (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(100)
);

INSERT INTO users_testing (name) VALUES ('Bob'), ('Alice');
```

### 🔹 Étape 3 : Créer le Dockerfile

Créer un fichier `Dockerfile` avec le contenu suivant :

```Dockerfile
FROM mysql:8

COPY init.sql /docker-entrypoint-initdb.d/

EXPOSE 3306
```

### 🔹 Étape 4 : Construire l’image MySQL

```bash
docker build -t daniel-mysql .
```

---

## Partie A : Utilisation SANS volume

### 🔹 Lancer le conteneur

```bash
docker run --name daniel-sql \
  -e MYSQL_ROOT_PASSWORD=rootM2dsia \
  -e MYSQL_DATABASE=testdb \
  -p 3307:3306 \
  -d daniel-mysql
```

### 🔹 Vérifier les données dans MySQL

```bash
docker exec -it daniel-sql mysql -u root -p -e "SELECT * FROM testdb.users_testing;"
```

(Mot de passe : `rootM2dsia`)

### 🔹 Supprimer le conteneur

```bash
docker stop daniel-sql
docker rm daniel-sql
```

---

## Partie B : Utilisation AVEC volume

### 🔹 Créer le volume

```bash
docker volume create daniel_volume
```

### 🔹 Lancer le conteneur avec volume

```bash
docker run --name daniel-sql-vol \
  -e MYSQL_ROOT_PASSWORD=rootM2dsia \
  -e MYSQL_DATABASE=testdb \
  -p 3308:3306 \
  -v daniel_volume:/var/lib/mysql \
  -d daniel-mysql
```

### 🔹 Insérer une nouvelle donnée

```bash
docker exec -it daniel-sql-vol mysql -u root -p -e "INSERT INTO testdb.users_testing(name) VALUES ('Charlie');"
```

### 🔹 Redémarrer et vérifier la persistance

```bash
docker stop daniel-sql-vol
docker start daniel-sql-vol
docker exec -it daniel-sql-vol mysql -u root -p -e "SELECT * FROM testdb.users_testing;"
```

---

## 🧹 Nettoyage (si nécessaire)

```bash
docker stop daniel-sql-vol
docker rm daniel-sql-vol
docker volume rm daniel_volume
docker rmi daniel-mysql
```
